# attention based seq2seq 로 summarization 구현
이상헌 <br>
voithru<br>

## 0. dataset 구현

마땅히 알맞은 데이터 셋을 찾지 못하다가 <br>

딥마인드의 [해당 논문](https://arxiv.org/abs/1506.03340) 에서 사용한 데이터셋은 **DMQA** 이다. <br>

여기서는 QnA의 모델링으로 사용하였는데, 해당 파일 안에 summary도 있나보다. <br>

### 0.1. 다운로드 및 밑작업
일단 다운로드를 시작한다. <br>
다음과 같은 [위치](http://cs.nyu.edu/~kcho/DMQA//)에서 받는다. <br>
```
CNN_stories.tgz
dailymail_stories.tgz
```
now unzip it <br>

이 안에는 @highlight로 emphasis 되어있는데, 이게 summary 라고 하는 것이다. <br>

밑작업을 위해서는 해당 데이터를 **tokenize**, 그리고 **binary** 로 만들어야 할 필요가 있다. <br>

cf) <br>
tokenize는 무엇인가. <br>
![tokenize 설명 img](https://cloud.githubusercontent.com/assets/2272790/18410099/1d0a1c1a-7761-11e6-9fe1-bd2e5622b90a.png)
* GO - ```<start>``` token. decoder에 가장 첫 번째 node에 들어갈 token.
* EOS - ```<end>``` token.
* UNK - unkown token. vocab에 들어있지 않는 rare vocab을 replace하기 위한 token. 우리 seq2seq에서는 사용되지 않는다! 하지만 굉장히 더러운 data에 있어서는 사용해야 한다. 예) ```my name is skdy33``` => ```my name is _unk_```
* PAD - mini-batch 안에 들어있는 데이터는 같은 길이를 가져야 한다. 따라서 짧은 데이터는 ```_pad_``` 토큰이 뒤에 붙는다.


### 참고자료
* [google's text summarization code](https://github.com/pranay360/TextSum_Data_Generation)
* [data-generation code](https://github.com/pranay360/TextSum_Data_Generation)
* [Get to the point : summarization with  pointer-generator networks](https://arxiv.org/pdf/1704.04368.pdf)
* [what does PAD / GO / EOS / UNK mean?](https://github.com/nicolas-ivanov/tf_seq2seq_chatbot/issues/15)
* [utf8 and encoding](wholetext=re.sub(r'[^\x00-\x7F]+','', wholetext)

In [1]:
# data conversion 시작

# 가장 먼저 nltk 설치하고 오세요
# sudo apt-get install python-nltk
# pip install nltk

import os
import re
import sys
import struct
import numpy as np
import collections
from nltk.tokenize import sent_tokenize
from tensorflow.core.example import example_pb2 # why do we need this
import struct # 이건 또 어떻게 사용하는 거지. https://docs.python.org/3/library/struct.html

In [2]:
## 또한 punkt corpus를 필요로 하는데, 왜 필요로 하는지는 모르겠지만 설치하자.
# 실제로 문장을 어디서 자를 지, 미리 학습된 tokenizer.
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/voithru/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 0.2 목적

일단 string을 binary로 만들어야 함은 자명하다. <br>
이러한 핸들링을 위하여 tensorflow는 ```from tensorflow.core.example import example_pb2``` 를 제공한다. <br>

본 형태는 다음과 같다. 
```python
features {
  feature {
    key: "abstract"
    value {
      bytes_list {
        value: "<d> <p> <s> president obama has moved this month \'s g - 8 summit from chicago to camp david </s> </p> </d>"
      }
    }
  }
  feature {
    key: "article"
    value {
      bytes_list {
        value: "<d> <p> <s> ( cnn ) - - when the white house , back in march , made the unexpected announcement that this month \'s g - 8 summit , long planned to take place in chicago , was being moved to camp david , the reasons given were bland: camp david was taking over from chicago \" to facilitate a free - flowing discussion with our close g - 8 partners , \" the white house announced . </s> <s> president obama was said to believe that camp david was a more intimate setting for the world leaders to get together . </s> <s> there was immediate conversation about the explanation . </s> <s> past g - 8 summits have been known to attract protests that culminated in violence in the streets of the cities where they have been held . </s> <s> camp david - - the presidential retreat in catoctin mountain park in frederick county , maryland - - is about as secure a sealed - off location as exists in this country . </s> <s> no one gets near it who is not supposed to get near it . </s> <s> originally the chicago g - 8 summit was supposed to be immediately followed in chicago by a separate summit of the leaders of nato nations . </s> <s> the nato meeting is still scheduled for chicago; like the g - 8 meeting , it will be hosted by obama . </s> <s> demonstrations are planned , and heavy security will be in place with the intention of keeping protesters well away from where the nato leaders are convening . </s> <s> increasingly , over the years , cities that engage in grand talk about \" bringing the world \" to see their many civic and cultural attributes end up going into a defensive crouch at the same time they are welcoming their global visitors . </s> </p> </d>"
      }
    }
  }
}
```
여기서 ```<d>,<p>,<s>``` 등의 tag를 보자. <br>
* ```<d>``` 의 경우는 document를 의미한다.
* ```<p>``` 의 경우는 paragrapgh를 의미한다.
* ```<s>``` 의 경우는 sentence를 의미한다.

여기서 큰 문제가 발생한다. <br>

## period가 없으면 sentence 단위로 문장을 끊는 것이 매우매우 어렵다.
실제로 nltk의 sent_tokenizer는 ```PunktSentenceTokenizer``` 라는 것을 사용하는데, <br>
이는 unsupervised training model로 어디서 문장을 끊을 지 학습한다. <br>

혹시 이를 학습시켜서 punctuation이 없는 도레에도 splitting을 시킬 수 있지 않을까?

* [how to split long sentences](http://www.aje.com/en/arc/editing-tip-strategies-splitting-long-sentences/)
* [PunktSentenceTokenizer](http://nlpforhackers.io/splitting-text-into-sentences/)
* [case1](https://kheafield.com/code/kenlm/)
* [case2](https://code.google.com/archive/p/berkeleylm/)
* [case3](http://www.speech.sri.com/projects/srilm/)

### 일단 이 issue 는 클로즈 한다. 영진, 종민에게 토스.

### 일단 CNN으로만 진행한다.

In [5]:
# word dictionary를 위한 counter 객체
counter = collections.Counter()
temp=0
#train, test, validation split
tr_r=0.85
v_r=0.08
# directory list
files = os.listdir('data/cnn/stories/')
# file 수
n_files = len(files)
print(n_files)
# train
train=files[:int(n_files*0.8)]
validation=files[len(train):len(train)+int(n_files*0.12)]
test=files[len(train)+len(validation):]

92579


In [7]:

    print(txt)

(cnn) -- the first anniversary of the egyptian revolution is today. egyptian society and the forces in egypt are in a state of anticipation. world media has its cameras and correspondents in cairo and major cities around the country. but many egyptians wonder if the revolution amounted to nothing more than a military coup.

it has been a year since the eruption of the first egyptian revolution that stunned the world and ended 30 years of authoritarian, oppressive and corrupt rule by hosni mubarak, egypt's last pharaoh. mubarak ended up in jail along with his sons and his regime's major figures, with stories of their unimaginable corruption, brutality and looting surfacing ever since.

but since then, most egyptians have become angry and frustrated with the performance of the military council -- comprising more than a dozen elderly generals -- that has taken control. egyptians, political forces and revolutionaries accuse the military council of being accomplices with the remnants of the

In [24]:
def convert_text2bin1(docs, writer):
    """
    CNN의 데이터를 <s>, <p>, <d> 의 알맞은 태그와 여러 input을 고정시켜주는 함수.
    input : 
        docs : list. list of raw data files
        writer : open buffer
    """
    global counter
    for i, fi in enumerate(docs):
        with open("data/cnn/stories/" + files[0],'r') as f:
            # decode 맞춰야 하나?
            txt = f.read().lower()
            # delete non-ascii
            txt = re.sub(r'[^\x00-\x7F]+','',txt)
            # '안녕' 을 ' 안녕 ' 으로 만든다.
            txt=re.sub(r"(\s?[\']\s+|\s+[\']\s?)"," ' ", txt)
            # "안녕" 을 " 안녕 " 으로 만든다.
            wholetext=re.sub(r'(\s?[\"]\s+|\s+[\"]\s?)',' " ', txt)
            # asdf's 를 asdf 's 로 바꿈
            wholetext=re.sub(r"(\'[s]\s+)"," 's ", wholetext)
            wholetext=wholetext.replace("."," . ")
            wholetext=wholetext.replace(","," , ")
            wholetext=wholetext.replace('-',' - ')
            wholetext=wholetext.replace('?',' ? ')
            wholetext=wholetext.replace('(','( ')
            wholetext=wholetext.replace(')',' )')
            data=wholetext.split("@highlight")
            # http://pavel.surmenok.com/2016/10/15/how-to-run-text-summarization-with-tensorflow/
            # 여기서 example_pb2 가 뭔지 잘 이해해야 할 것 같다.
            # 그리고 여기서는 하나의 highlight만을 사용한다. 왜 하나의 highlight만을 사용하지?
            news=data[0]
            highlights=data[1].replace('\n\n','')
            news=(" ".join(news.split('\n\n'))).strip()
            sentences = sent_tokenize(news) # 여기서 tokenize는 
            news = '<d> <p> ' + ' '.join(['<s> ' + sentence + ' </s>' for sentence in sentences]) + ' </p> </d>'
            highlights = '<d> <p> <s> ' + highlights + ' </s> </p> </d>'
            words = (news+" "+highlights).split()
            counter.update(words)
            tf_example = example_pb2.Example()
            tf_example.features.feature['article'].bytes_list.value.extend([(' '.join(news.split())).encode('utf-8')])
            tf_example.features.feature['abstract'].bytes_list.value.extend([(' '.join(highlights.split())).encode('utf-8')])
            tf_example_str = tf_example.SerializeToString() # 이게 format을 binary 화 한 것.
            str_len = len(tf_example_str)
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, tf_example_str)) # this means str-len length string.
            if i%3000 == 0:
                print(int((float(i)/ len(docs))*100), "%")
    print((float(len(docs))/ len(docs))*100, "%...." "converted")

In [25]:
with open('data/trainCNN.bin','wb') as writer:
    convert_text2bin1(train,writer)

0 %
4 %
8 %
12 %
16 %
20 %
24 %
28 %
32 %
36 %
40 %
44 %
48 %
52 %
56 %
60 %
64 %
68 %
72 %
76 %
81 %
85 %
89 %
93 %
97 %
100.0 %....converted


In [26]:
with open('data/testCNN.bin','wb') as writer:
    convert_text2bin1(test,writer)
with open('data/validateCNN.bin','wb') as writer:
    convert_text2bin1(validation,writer)

0 %
40 %
81 %
100.0 %....converted
0 %
27 %
54 %
81 %
100.0 %....converted


In [28]:
# 이제 vocabulary를 파일로 만들어서 넣어야 한다. 여기서 갯수는 200000-2 로 한 이유는 잘 모르겠고
# 또한 이 데이터는 굳이 binary로 할 이유가 없다.
mc = counter.most_common(200000-2)

In [30]:
with open("data/vocab", 'w') as writer:
    for word, count in mc:
        writer.write(word + " " + str(count) + '\n')
    writer.write('<UNK> 0\n') # 언노운 태그
    writer.write('<PAD> 0\n') # 패딩 tag.

## 이제 논문구현

현재 구글 [github 파일](https://github.com/tensorflow/models/tree/master/research/textsum)에서는
* build 는 bazel로
* [data batcher](https://github.com/tensorflow/models/blob/master/research/textsum/data.py)
* [batch reader](https://github.com/tensorflow/models/blob/master/research/textsum/batch_reader.py)
* [beam search](https://github.com/tensorflow/models/blob/master/research/textsum/beam_search.py)
* [seq2seq-attention model](https://github.com/tensorflow/models/blob/master/research/textsum/seq2seq_attention_model.py)
* [seq2seq-attention decoding](https://github.com/tensorflow/models/blob/master/research/textsum/seq2seq_attention_decode.py)
* [seq2seq-attention](https://github.com/tensorflow/models/blob/master/research/textsum/seq2seq_attention.py)
* [seq2seq-attention-lib](https://github.com/tensorflow/models/blob/master/research/textsum/seq2seq_lib.py)

으로 나뉘어 있다. <br>

먼저 스키밍을 통하여 각자의 역할을 알아보도록 한다.